In [205]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import random
import joblib
import pandas as pd

In [206]:
# Node labels
farms = ["F"+str(i) for i in range(1,31)]
custs = ["C"+str(i) for i in range(1,33)]
prods = ['P1', 'P2', 'P3']
dealer = "Dealer"

In [207]:
G = nx.DiGraph()

In [208]:
# Add nodes to network graph
G.add_nodes_from(farms)
G.add_node(dealer)
G.add_nodes_from(prods)
G.add_nodes_from(custs)

In [209]:
farm_dealer_edge_tuples =[(f,dealer) for f in farms]
dealer_prod_edge_tuples =[(dealer, p) for p in prods]
prod_cust_tuples = [(p,c) for p in prods for c in custs]

In [210]:
edges = farm_dealer_edge_tuples + dealer_prod_edge_tuples + prod_cust_tuples

In [211]:
# Connect nodes with edges
G.add_edges_from(edges)

In [212]:
# Set the farm categories
for i, f in enumerate(farms, start=1):
    c = ''
    if i <=3:
        c = 'Large'
    elif 3 < i < 10:
        c = 'Medium'
    else:
        c = 'Small'
        
    G.nodes[f]['category']=c


Large farms supply 100,000 eggs/day<br>
Medium farms supply 500,000 eggs/day<br>
Small farms supply 10,000 eggs/day<br>

In [213]:
for i, f in enumerate(farms, start=1):
    z = 0
    if G.nodes[f]['category'] == 'Large':
        z = 100000
    elif G.nodes[f]['category'] == 'Medium':
        z = 50000
    else:
        z = 10000
    
    G[f]['Dealer']['quantity']=z

The cost/egg from Large farms = \$ 0.22<br>
The cost/egg from Medium farms = \$ 0.23<br>
The cost/egg from Small farms = \$ 0.26<br>

In [214]:
# Add prices per egg charged by farms 
for farm in farms:
    if G.nodes[farm]['category']=='Large':
        per_egg = 0.22
    elif G.nodes[farm]['category']=='Medium':
        per_egg = 0.23
    else:
        per_egg = 0.26

    G[farm]['Dealer']['cost_per_egg'] = per_egg 
    

In [215]:
# Set the type of product
G.nodes['P1']['eggs_per_box'] = 6
G.nodes['P2']['eggs_per_box'] = 10
G.nodes['P3']['eggs_per_box'] = 12

In [216]:
# Set the category of customer

for i, cust in enumerate(custs,start=1):
    if i <= 6:
        cat = 'Large'
    elif 6 < i <=12:
        cat = 'Medium'
    else:
        cat = 'Small'
    
    G.nodes[cust]['category'] = cat

In [217]:
# Set the location of customers

locs =['South', 'North', 'South', 'North', 'South', 'North', 'North', 'South', 'South', 'North', 'North', 'North', 'North', 'South', 'North', 'North', 'North', 'South', 'South', 'South', 'North', 'North', 'South', 'North', 'North', 'South', 'South', 'South', 'South', 'North', 'North', 'North']

for cust, l in zip(custs, locs):
    G.nodes[cust]['location'] = l

In [262]:
# Get the data from excel
dxls = pd.ExcelFile('data.xlsx')
demand = pd.read_execel(xls, sheet_name='demand', index_col=0, usecols="A:D", nrows=33)
prices = dpd.read_excel(xls, sheet_name = 'prices', index_col=0, usecols="A:D", nrows=33)

In [277]:
# Set the demand for products from excel data 
for cust in custs:
    for prod in G.predecessors(cust):
        G[prod][cust]['demand'] = demand.loc[cust][prod] 

In [278]:
# Set the prices of products from excel data
for cust in custs:
    for prod in G.predecessors(cust):
        G[prod][cust]['price'] = prices.loc[cust][prod] 

In [281]:
# Set the transport costs per egg based on location
transport_cost_per_egg = {'North':0.10, 'South':0.15}

In [282]:
# Save the graph as a joblib object
joblib.dump(G, 'supply_chain_graph')
joblib.dump(farms, 'farms_list')
joblib.dump(custs, 'customer_list')
joblib.dump(prods, 'product_list')
joblib.dump(transport_cost_per_egg, 'transport_costs')

['transport_costs']

## Fill the large orders first 

In [304]:
category_order = ['Large', 'Medium', 'Small']

In [305]:
from pso import get_total_supply

In [306]:
supply = get_total_supply(G)
supply

810000

In [295]:
def fill_order(order_list: list, graph: nx.DiGraph) -> np.ndarray:
    ''' Returns the quantities boxes in the order of being filled by order_list given supply constraints'''
    global prods, G, supply
    ordered_tuples = [(p,cust) for co in order_list for p in prods for cust in graph.successors(p) if graph.nodes[cust]['category']==co]
    supply_copy = supply.copy()
    zero_array = np.zeros(len(ordered_tuples))
    for i, ((p, c), z) in enumerate(zip(ordered_tuples, zero_array)):
        if graph[p][c]['demand'] ==0:
            continue
        else:
            if supply_copy > graph.nodes[p]['eggs_per_box']:
                total_supply_boxes = int(supply_copy/graph.nodes[p]['eggs_per_box'])
                if total_supply_boxes > graph[p][c]['demand']:
                    zero_array[i] = graph[p][c]['demand']
                    supply_copy -= graph[p][c]['demand'] * graph.nodes[p]['eggs_per_box']
                elif total_supply_boxes < graph[p][c]['demand']:
                    zero_array[i] = total_supply_boxes
                    supply_copy -= total_supply_boxes * graph.nodes[p]['eggs_per_box']
            else:
                break
    return zero_array

In [301]:
q_vec = fill_order(order_list=category_order, graph=G)
q_vec

array([  20.,   80.,   40.,  100.,  100.,   40.,   80.,   20.,   20.,
         80.,   40.,  100.,   80.,   40.,   20.,  100.,   20.,   80.,
         80.,   60.,   40.,   40.,   40.,   40.,   40.,   40.,   40.,
         40.,   40.,   40.,   40.,   40.,   40.,   40.,   40.,    0.,
          0.,    0.,    0.,    0.,   30.,   30.,   30.,   30.,   30.,
         30.,   30.,   30.,   30.,   30.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0., 3000.,  500.,  500.,
        500.,  500., 1000., 1000.,  200.,  300.,  300.,  200., 1000.,
        200.,   50.,   50.,  150.,   50.,  100., 6000., 5000., 6000.,
       5000., 6000., 5000., 6000., 3000., 5000., 4000., 6000., 3000.,
       3000., 4000., 5000., 2000., 4000.,    0.])

In [302]:
def calc_profit(qty_vec: np.ndarray, order_list: list, graph: nx.DiGraph):
    ''' Takes a vector of product quantities (qty_vec) and the returns the profit/loss of that configuration'''
    global prods, G, transport_cost_per_egg
    ordered_tuples = [(p,cust) for co in order_list for p in prods for cust in G.successors(p) if G.nodes[cust]['category']==co]
    avg_cost_per_egg = np.mean([graph[farm]['Dealer']['cost_per_egg'] for farm in graph.predecessors('Dealer')])
    total_eggs = np.sum([qv * graph.nodes[p]['eggs_per_box'] for (p,c), qv  in zip(ordered_tuples, qty_vec)])
    total_egg_cost = avg_cost_per_egg * total_eggs 
    total_sales = np.sum([G[p][c]['price'] * qv for (p,c), qv in zip(ordered_tuples, qty_vec)])
    transport_cost = np.sum([transport_cost_per_egg[graph.nodes[c]['location']] * graph.nodes[p]['eggs_per_box'] * qv  for (p,c), qv in zip(ordered_tuples, qty_vec)])
    return total_sales - (transport_cost+ total_egg_cost) 

In [303]:
calc_profit(qty_vec=q_vec, order_list=category_order, graph=G)

619946.0